In [1]:
import sys
sys.executable

'/Users/kevin/anaconda3/envs/tf_env/bin/python'

### Get Python driver version the same as the Pyspark worker
#### Pyspark needs both driver and worker nodes to use the same Python version

In [2]:
import os
os.environ['PYSPARK_PYTHON'] = '/Users/kevin/anaconda3/envs/tf_env/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/Users/kevin/anaconda3/envs/tf_env/bin/python'

# Data Preprocessing

### 1) Load in relevant datasets pertaining to nuclear energy in 2023
#### deuterium derived by country, tritium derived by country, rare earth metals derived by country, deuterium reserves by country, tritium reserves by country, rare earth reserves by country, nuclear energy capacity by countries that have already embraced nuclear, actual nuclear energy supplied by country, (operating nuclear reactors by country?) 
### 2) Merge all of these metrics into one dataset for training and testing of various machine learning models
### 3) Use the trained models on a dataset of countries that have not yet embraced nuclear but they have deuterium, tritium and rare earths (either derived or reserves)
### Use confidence intervals to gauge an estimate of how much nuclear energy theses countries can supply

### Remove empty columns/rows, merge datasets into one dataframe for training and testing 
### match up countries 

In [3]:
pip install pandas beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pyspark.sql import SparkSession

In [5]:
spark= SparkSession.builder \
        .appName("Pyspark SQL in Jupyter") \
        .getOrCreate()

24/08/22 11:33:02 WARN Utils: Your hostname, Kevins-MacBook-Pro-97.local resolves to a loopback address: 127.0.0.1; using 10.1.10.100 instead (on interface en0)
24/08/22 11:33:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/22 11:33:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Scraping

## Makes a request at the PRIS IAEA URL, parses html to find and read table, to finally turn it into a csv

In [6]:
url= "https://pris.iaea.org/PRIS/WorldStatistics/NuclearShareofElectricityGeneration.aspx"
response=requests.get(url)
soup=BeautifulSoup(response.text, 'html.parser')

table=soup.find('table')

df=pd.read_html(str(table))[0]

df.to_csv('nuclear_share_of_electricity_generation_2023', index=False)

# Total Net Electrical Capacity (MW), Number of Operated Reactors, Nuclear Electricity Supplied (GW.h), Nuclear share % by country

## Using 2023 data because provides net electrical capacity as well as nucelar electricity supplied

### These actual values will help the models make predictions

In [7]:
nuclear_share_of_electricity_generation_2023 = spark.read.csv('nuclear_share_of_electricity_generation_2023', header=True, inferSchema=True)
nuclear_share_of_electricity_generation_2023.show()

+--------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|             Country|Total Net Electrical Capacity [MW]|Number of Operated Reactors|Nuclear Electricity Supplied  [GW.h]|Nuclear Share  [%]|
+--------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|              FRANCE|                             61370|                         56|                           323773.23|              64.8|
|            SLOVAKIA|                              2308|                          5|                            17004.98|              61.3|
|             HUNGARY|                              1916|                          4|                            15091.64|              48.8|
|             FINLAND|                              4394|                          5|                            32759.35|              42.0|
|     

## Observe exact column names

In [8]:
print(nuclear_share_of_electricity_generation_2023.columns)

['Country', 'Total Net Electrical Capacity [MW]', 'Number of Operated Reactors', 'Nuclear Electricity Supplied  [GW.h]', 'Nuclear Share  [%]']


## Reorder table based on Nuclear Electricity Supplied

In [9]:
nuclear_share_of_electricity_generation_2023.createOrReplaceTempView("nuclear_share")


# SQL query ; Select all from nuclear share table, order by the nuclear electricity supplied column in descending order
ordered_nuclear_electricity= spark.sql("""
SELECT * 
FROM nuclear_share 
ORDER BY `Nuclear Electricity Supplied  [GW.h]` DESC
""")

ordered_nuclear_electricity.show()

+--------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|             Country|Total Net Electrical Capacity [MW]|Number of Operated Reactors|Nuclear Electricity Supplied  [GW.h]|Nuclear Share  [%]|
+--------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|               Total|                            364480|                        403|                          2552067.11|              null|
|UNITED STATES OF ...|                             95835|                         93|                           779186.02|              18.6|
|               CHINA|                             53152|                         55|                           406483.53|               4.9|
|              FRANCE|                             61370|                         56|                           323773.23|              64.8|
|     

## Remove row displaying "total"

In [10]:
# Filters out rows where "Country" is "Total" 
filtered_df = ordered_nuclear_electricity.filter(~ordered_nuclear_electricity["Country"].like("%Total%"))
filtered_df.createOrReplaceTempView("filtered_df")
filtered_df.show()

+--------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|             Country|Total Net Electrical Capacity [MW]|Number of Operated Reactors|Nuclear Electricity Supplied  [GW.h]|Nuclear Share  [%]|
+--------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|UNITED STATES OF ...|                             95835|                         93|                           779186.02|              18.6|
|               CHINA|                             53152|                         55|                           406483.53|               4.9|
|              FRANCE|                             61370|                         56|                           323773.23|              64.8|
|              RUSSIA|                             27727|                         37|                           203957.32|              18.4|
|  KOR

### Get current count of rows

In [11]:
filtered_df.count()

31

## Add info on Taiwan since it was excluded from the original table, adds to our training and testing data

In [12]:
from pyspark.sql import Row

# define row object
# When using ** before a dictionary in a function call or object creation, ** unpacks the dictionary into keyword arguments
# allows to pass dictionary keys as parameter names and their values as corresponding argument values

taiwan_row = Row(
    Country = "TAIWAN, CHINA",
    # Using dictionary syntax to handle column names with spaces
    **{"Total Net Electrical Capacity [MW]": 2859},
    **{"Number of Operated Reactors": 3}, 
    **{"Nuclear Electricity Supplied  [GW.h]": 17153.88}, 
    **{"Nuclear Share  [%]": 6.9}, 
)

# Convert row into pd dataframe
taiwan_row_df = spark.createDataFrame([taiwan_row])

# create temp view for new row
taiwan_row_df.createOrReplaceTempView("taiwan_row_view")

### Combine row into table

In [13]:
combined_df= spark.sql("""
SELECT * FROM filtered_df
UNION ALL
SELECT * FROM taiwan_row_view
""")

combined_df.show()

+--------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|             Country|Total Net Electrical Capacity [MW]|Number of Operated Reactors|Nuclear Electricity Supplied  [GW.h]|Nuclear Share  [%]|
+--------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|UNITED STATES OF ...|                             95835|                         93|                           779186.02|              18.6|
|               CHINA|                             53152|                         55|                           406483.53|               4.9|
|              FRANCE|                             61370|                         56|                           323773.23|              64.8|
|              RUSSIA|                             27727|                         37|                           203957.32|              18.4|
|  KOR

In [14]:
# confirm if Taiwan was successfully added
rows = combined_df.count()
combined_df.show(rows, truncate=False)

+---------------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|Country                    |Total Net Electrical Capacity [MW]|Number of Operated Reactors|Nuclear Electricity Supplied  [GW.h]|Nuclear Share  [%]|
+---------------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|UNITED STATES OF AMERICA   |95835                             |93                         |779186.02                           |18.6              |
|CHINA                      |53152                             |55                         |406483.53                           |4.9               |
|FRANCE                     |61370                             |56                         |323773.23                           |64.8              |
|RUSSIA                     |27727                             |37                         |203957.32     

## Order based on Nuclear Energy Supplied [GW.h] again with new Taiwan row

In [15]:
combined_df.createOrReplaceTempView("combined_filtered_df")

final_ordered_nuclear_electricity=spark.sql("""
SELECT *
FROM combined_filtered_df
ORDER BY `Nuclear Electricity Supplied  [GW.h]`
DESC
""")

final_ordered_nuclear_electricity.show(32, truncate=False)

+---------------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|Country                    |Total Net Electrical Capacity [MW]|Number of Operated Reactors|Nuclear Electricity Supplied  [GW.h]|Nuclear Share  [%]|
+---------------------------+----------------------------------+---------------------------+------------------------------------+------------------+
|UNITED STATES OF AMERICA   |95835                             |93                         |779186.02                           |18.6              |
|CHINA                      |53152                             |55                         |406483.53                           |4.9               |
|FRANCE                     |61370                             |56                         |323773.23                           |64.8              |
|RUSSIA                     |27727                             |37                         |203957.32     

# Lithium dataset
## MCS 2024 Provides information on Country, extraction type, true production in 2022, estimated production in 2023, production notes, reserves in tons, and reserves notes
### The Mineral Commodities Summary withholds information on the United States to avoid disclosing company proprietary data 

In [16]:
lithium_2024 = pd.read_csv('mcs2024-lithi_world.csv')
lithium_2024

,Source,Country,Type,Prod_t_2022,Prod_t_est_2023,Prod_notes,Reserves_t,Reserves_notes,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,MCS2024,United States,"Mine production, lithium content",W,W,NaN,1100000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MCS2024,Argentina,"Mine production, lithium content",6590,9600,NaN,3600000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MCS2024,Australia,"Mine production, lithium content",74700,86000,NaN,6200000.0,"For Australia, Joint Ore Reserves Committee-co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MCS2024,Brazil,"Mine production, lithium content",2630,4900,Also estimated in 2022,390000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MCS2024,Canada,"Mine production, lithium content",520,3400,Also estimated in 2022,930000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MCS2024,Chile,"Mine production, lithium content",38000,44000,NaN,9300000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MCS2024,China,"Mine production, lithium content",22600,33000,Also estimated in 2022,3000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MCS2024,Portugal,"Mine production, lithium content",380,380,Also estimated in 2022,60000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MCS2024,Zimbabwe,"Mine production, lithium content",1030,3400,Also estimated in 2022,310000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MCS2024,Other countries6,"Mine production, lithium content",0,0,NaN,2800000.0,"Reserves for Argentina, Australia, Brazil, Chi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Remove empty rows and columns

### Remove empty columns

In [17]:
# if the index of the column is greater than the index of 'Reserves_notes' (the last unproblematic column) then we add it to the remove list
# columns.values is just the names of each of the columns, then we get the index based on the name as the key using columns.get_loc
# then we compare indices
# finally drop the remove list with axis=1 (columns)

remove_cols = [i for i in lithium_2024.columns.values if lithium_2024.columns.get_loc(i)  > lithium_2024.columns.get_loc('Reserves_notes')]

lithium_2024_no_bad_cols = lithium_2024.drop(remove_cols, axis=1)
lithium_2024_no_bad_cols

,Source,Country,Type,Prod_t_2022,Prod_t_est_2023,Prod_notes,Reserves_t,Reserves_notes
0,MCS2024,United States,"Mine production, lithium content",W,W,NaN,1100000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
1,MCS2024,Argentina,"Mine production, lithium content",6590,9600,NaN,3600000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
2,MCS2024,Australia,"Mine production, lithium content",74700,86000,NaN,6200000.0,"For Australia, Joint Ore Reserves Committee-co..."
3,MCS2024,Brazil,"Mine production, lithium content",2630,4900,Also estimated in 2022,390000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
4,MCS2024,Canada,"Mine production, lithium content",520,3400,Also estimated in 2022,930000.0,NaN
5,MCS2024,Chile,"Mine production, lithium content",38000,44000,NaN,9300000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
6,MCS2024,China,"Mine production, lithium content",22600,33000,Also estimated in 2022,3000000.0,NaN
7,MCS2024,Portugal,"Mine production, lithium content",380,380,Also estimated in 2022,60000.0,NaN
8,MCS2024,Zimbabwe,"Mine production, lithium content",1030,3400,Also estimated in 2022,310000.0,NaN
9,MCS2024,Other countries6,"Mine production, lithium content",0,0,NaN,2800000.0,"Reserves for Argentina, Australia, Brazil, Chi..."


### Remove empty rows

In [18]:
# same procedure; problematic rows start after row 10, so if row number is greater than 10, we will remove 

remove_rows = (i for i in range(11, lithium_2024_no_bad_cols.shape[0]))
# .shape gives the dimensions of the data in format (rows, columns) 
# example: lithium_2024_no_bad_colss has shape (26,8) 26 rows, 8 columns

lithium_2024_no_bad_cols_rows = lithium_2024_no_bad_cols.drop(remove_rows, axis=0)
lithium_2024_no_bad_cols_rows

,Source,Country,Type,Prod_t_2022,Prod_t_est_2023,Prod_notes,Reserves_t,Reserves_notes
0,MCS2024,United States,"Mine production, lithium content",W,W,NaN,1100000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
1,MCS2024,Argentina,"Mine production, lithium content",6590,9600,NaN,3600000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
2,MCS2024,Australia,"Mine production, lithium content",74700,86000,NaN,6200000.0,"For Australia, Joint Ore Reserves Committee-co..."
3,MCS2024,Brazil,"Mine production, lithium content",2630,4900,Also estimated in 2022,390000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
4,MCS2024,Canada,"Mine production, lithium content",520,3400,Also estimated in 2022,930000.0,NaN
5,MCS2024,Chile,"Mine production, lithium content",38000,44000,NaN,9300000.0,"Reserves for Argentina, Australia, Brazil, Chi..."
6,MCS2024,China,"Mine production, lithium content",22600,33000,Also estimated in 2022,3000000.0,NaN
7,MCS2024,Portugal,"Mine production, lithium content",380,380,Also estimated in 2022,60000.0,NaN
8,MCS2024,Zimbabwe,"Mine production, lithium content",1030,3400,Also estimated in 2022,310000.0,NaN
9,MCS2024,Other countries6,"Mine production, lithium content",0,0,NaN,2800000.0,"Reserves for Argentina, Australia, Brazil, Chi..."


## Remove columns not useful to the analysis

### For the purpose of our analysis, we don't need columns: Source, Type, Prod Notes, Reserves Notes

In [19]:
remove_unnecessary = ['Source',
                      'Type',
                      'Prod_notes',
                      'Reserves_notes'
                     ]
lithium_2024_clean = lithium_2024_no_bad_cols_rows.drop(remove_unnecessary, axis=1)
lithium_2024_clean

,Country,Prod_t_2022,Prod_t_est_2023,Reserves_t
0,United States,W,W,1100000.0
1,Argentina,6590,9600,3600000.0
2,Australia,74700,86000,6200000.0
3,Brazil,2630,4900,390000.0
4,Canada,520,3400,930000.0
5,Chile,38000,44000,9300000.0
6,China,22600,33000,3000000.0
7,Portugal,380,380,60000.0
8,Zimbabwe,1030,3400,310000.0
9,Other countries6,0,0,2800000.0


# Rare Earth Ores dataset

# Since its from the mcs as well, provides info in the same format as the Lithium dataset
### All preprocessing will be done in one block

In [20]:
rareearth_2024=pd.read_csv('mcs2024-raree_world.csv')
rareearth_2024

,Source,Country,Type,Prod_t_est_2022,Prod_t_est_2023,Prod_notes,Reserves_t,Reserves_notes,Unnamed: 8,Unnamed: 9,...,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43
0,MCS2024,United States,"Rare earths, mine production, rare-earth-oxide...",42000.0,43000.0,NaN,1800000.0,"Reserves for Australia, Russia, Thailand, and ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MCS2024,Australia,"Rare earths, mine production, rare-earth-oxide...",18000.0,18000.0,NaN,5700000.0,"For Australia, Joint Ore Reserves Committee-co...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MCS2024,Brazil,"Rare earths, mine production, rare-earth-oxide...",80.0,80.0,NaN,21000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MCS2024,Burma,"Rare earths, mine production, rare-earth-oxide...",12000.0,38000.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MCS2024,Canada,"Rare earths, mine production, rare-earth-oxide...",0.0,0.0,NaN,830000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MCS2024,China,"Rare earths, mine production, rare-earth-oxide...",210000.0,240000.0,Production quota; does not include undocumente...,44000000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MCS2024,Greenland,"Rare earths, mine production, rare-earth-oxide...",0.0,0.0,NaN,1500000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MCS2024,India,"Rare earths, mine production, rare-earth-oxide...",2900.0,2900.0,NaN,6900000.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,MCS2024,Madagascar,"Rare earths, mine production, rare-earth-oxide...",960.0,960.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,MCS2024,Malaysia,"Rare earths, mine production, rare-earth-oxide...",80.0,80.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
remove_cols = [i for i in rareearth_2024.columns.values if rareearth_2024.columns.get_loc(i)  > rareearth_2024.columns.get_loc('Reserves_notes')]

rareearth_2024_no_bad_cols = rareearth_2024.drop(remove_cols, axis=1)

remove_rows = (i for i in range(16, rareearth_2024_no_bad_cols.shape[0]))

rareearth_2024_no_bad_cols_rows = rareearth_2024_no_bad_cols.drop(remove_rows, axis=0)

remove_unnecessary = ['Source',
                      'Type',
                      'Prod_notes',
                      'Reserves_notes'
                     ]

rareearth_2024_clean = rareearth_2024_no_bad_cols_rows.drop(remove_unnecessary, axis=1)
rareearth_2024_clean

,Country,Prod_t_est_2022,Prod_t_est_2023,Reserves_t
0,United States,42000.0,43000.0,1800000.0
1,Australia,18000.0,18000.0,5700000.0
2,Brazil,80.0,80.0,21000000.0
3,Burma,12000.0,38000.0,NaN
4,Canada,0.0,0.0,830000.0
5,China,210000.0,240000.0,44000000.0
6,Greenland,0.0,0.0,1500000.0
7,India,2900.0,2900.0,6900000.0
8,Madagascar,960.0,960.0,NaN
9,Malaysia,80.0,80.0,NaN


# Zirconium and Hafnium dataset

In [22]:
zirco_2024=pd.read_csv('mcs2024-zirco_world.csv')
zirco_2024

,Source,Country,Type,Prod_kt_est_2022,Prod_kt_est_2023,Prod_notes,Reserves_kt_2023,Reserves_notes
0,MCS2024,United States,"Zirconium ores and zircon concentrates, mine p...",100,100,Data are rounded to the nearest hundred thousa...,500.0,NaN
1,MCS2024,Australia,"Zirconium ores and zircon concentrates, mine p...",500,500,NaN,55000.0,"For Australia, Joint Ore Reserves Committee-co..."
2,MCS2024,China,"Zirconium ores and zircon concentrates, mine p...",140,140,NaN,72.0,"Zirconium reserves for Australia, China, Mozam..."
3,MCS2024,Indonesia,"Zirconium ores and zircon concentrates, mine p...",97,90,NaN,NaN,NaN
4,MCS2024,Kenya,"Zirconium ores and zircon concentrates, mine p...",27,30,NaN,18.0,NaN
5,MCS2024,Madagascar,"Zirconium ores and zircon concentrates, mine p...",27,30,NaN,2300.0,NaN
6,MCS2024,Mozambique,"Zirconium ores and zircon concentrates, mine p...",104,90,NaN,1500.0,"Zirconium reserves for Australia, China, Mozam..."
7,MCS2024,Senegal,"Zirconium ores and zircon concentrates, mine p...",57,50,NaN,2600.0,NaN
8,MCS2024,Sierra Leone,"Zirconium ores and zircon concentrates, mine p...",34,30,NaN,290.0,NaN
9,MCS2024,South Africa,"Zirconium ores and zircon concentrates, mine p...",300,400,NaN,5600.0,"Zirconium reserves for Australia, China, Mozam..."


# Since its from the mcs as well, provides info in the same format as the Lithium dataset
### All preprocessing will be done in one block

In [23]:
remove_unnecessary = ['Source',
                      'Type',
                      'Prod_notes',
                      'Reserves_notes'
                     ]

zirco_2024_clean = zirco_2024.drop(remove_unnecessary, axis=1)
zirco_2024_clean

,Country,Prod_kt_est_2022,Prod_kt_est_2023,Reserves_kt_2023
0,United States,100,100,500.0
1,Australia,500,500,55000.0
2,China,140,140,72.0
3,Indonesia,97,90,NaN
4,Kenya,27,30,18.0
5,Madagascar,27,30,2300.0
6,Mozambique,104,90,1500.0
7,Senegal,57,50,2600.0
8,Sierra Leone,34,30,290.0
9,South Africa,300,400,5600.0


#### Not using the salient files because they pertain to the US exclusively, and some information is withheld

# Deuterium dataset

## Using GNIP (Global Network of Isotopes in Precipitation) data from 2021-2024
## Too few data in just 2023 alone 
## Configurations: All default dataset categories, default sample types, 66 countries/territories,2 Measurands available (Deuterium in Water, Tritium), 121 datasets.
## 299 sample sites
## 4126 samples
## 4603 observations

# GNIP Plan:
## Filter out unnecessary columns
## Remove duplicate rows if any
## Summate relative values (H2= Deuterium, H3= Tritium)
### Site-Specific Aggregation; use mean median sum only for the same sample sites across time ; gives representative value across each site
### To estimate total amount of measurand within a country, sum the site-specific aggregated values
#### Aggregating by summation
### This approach assumes that the site-specific measurements represent independent sources or deposits, where aggregation by summation provides a cumulative total
## This format will simplify model training

In [26]:
# load and preview the data
deutinwater_tritium = pd.read_csv('GNIP_deutinwater_tritium.csv')
deutinwater_tritium.head()

,Sample UID,Observation UID,Sample Site UID,Dummy Sample,Circulation,Category Group Name,Project Number,Project Title,Project Region,Project Group Name,...,Measurand Amount,Measurand Uncertainty,Measurand Unit,Observation Date,Observation Date Accuracy,Field Preparation Method Name,Data Provider Code,Data Provider Name,Quality Flag Code,Quality Flag Name
0,590b1c7e-1d0c-4b4c-b53f-f6553b6ebe84,bcbf8b27-3f9d-4c4a-b8d7-2712b152ff4f,2f5eb618-1af3-4b54-9404-f79b77fe150c,False,Public,Precipitation,GNIP/M/US/16,GNIP/USA (UVU),USA (Orem),GNIP - Monthly,...,-125.4,NaN,‰ vs VSMOW-SLAP,NaN,None,NaN,1470.0,"Utah Valley University, Dept. of Earth Science",NaN,NaN
1,6f0ee52a-eec6-4e37-8ff8-c6507f6f5c78,907f8a8d-a046-40c5-a5bc-d7477d1ec511,2f5eb618-1af3-4b54-9404-f79b77fe150c,False,Public,Precipitation,GNIP/M/US/16,GNIP/USA (UVU),USA (Orem),GNIP - Monthly,...,-174.1,NaN,‰ vs VSMOW-SLAP,NaN,None,NaN,1470.0,"Utah Valley University, Dept. of Earth Science",NaN,NaN
2,9494aaba-6be1-499e-81f1-9f3a23d029df,b27b0fa5-629d-4393-a76b-f1a130ad3ee5,2f5eb618-1af3-4b54-9404-f79b77fe150c,False,Public,Precipitation,GNIP/M/US/16,GNIP/USA (UVU),USA (Orem),GNIP - Monthly,...,-100.1,NaN,‰ vs VSMOW-SLAP,NaN,None,NaN,1470.0,"Utah Valley University, Dept. of Earth Science",NaN,NaN
3,04edf493-af2e-4bf1-a522-0150d91179dc,edf47c35-4c3d-461e-b9c0-eb65fd31bd23,2f5eb618-1af3-4b54-9404-f79b77fe150c,False,Public,Precipitation,GNIP/M/US/16,GNIP/USA (UVU),USA (Orem),GNIP - Monthly,...,-77.3,NaN,‰ vs VSMOW-SLAP,NaN,None,NaN,1470.0,"Utah Valley University, Dept. of Earth Science",NaN,NaN
4,fabe3adc-54aa-4cc5-bc5e-60703af449c6,2095d79f-a9a9-4c07-9ba2-1553a9eb9d74,2f5eb618-1af3-4b54-9404-f79b77fe150c,False,Public,Precipitation,GNIP/M/US/16,GNIP/USA (UVU),USA (Orem),GNIP - Monthly,...,-48.2,NaN,‰ vs VSMOW-SLAP,NaN,None,NaN,1470.0,"Utah Valley University, Dept. of Earth Science",NaN,NaN


In [27]:
# observe column names for dimensionality reduction
deutinwater_tritium.columns.values

array(['Sample UID', 'Observation UID', 'Sample Site UID', 'Dummy Sample',
       'Circulation', 'Category Group Name', 'Project Number',
       'Project Title', 'Project Region', 'Project Group Name',
       'Campaign Name', 'Sample Site Name', 'Latitude', 'Longitude',
       'Altitude', 'WMO Code', 'Country ISO Code',
       'Sample Site Type Name', 'Sample Name', 'Sample Date',
       'Sample Date Accuracy', 'Lith Adj Code', 'Lith Adj Name',
       'Lith Noun Code', 'Lith Noun Name', 'Period From', 'Period To',
       'Min Depth', 'Max Depth', 'Sampling Method Code',
       'Sampling Method Name', 'Sample Media Type Code',
       'Sample Media Type Name', 'Sample Info', 'Measurand Symbol',
       'Observation Method Code', 'Observation Method Name',
       'Measurand Prefix', 'Measurand Amount', 'Measurand Uncertainty',
       'Measurand Unit', 'Observation Date', 'Observation Date Accuracy',
       'Field Preparation Method Name', 'Data Provider Code',
       'Data Provider Name', 

In [28]:
# Double check H2 and H3 are included in our measurands
deutinwater_tritium['Measurand Symbol'].unique()

array(['H2', 'H3'], dtype=object)

## Dimensionality reduction

In [39]:
gnip_unnecessary = ['Sample UID',
                    'Observation UID',
                    'Sample Site UID',
                    'Dummy Sample',
                    'Circulation',
                    'Category Group Name', # provides info on if is from precipitation or surface waters, but we want the total from all sources so we exclude
                    'Project Number',
                    'Project Title',
                    'Project Group Name',
                    'Campaign Name',
                    'Latitude',
                    'Longitude', # lat and long pertain to the sample site 
                    'Altitude', # altitude pertains to sample site 
                    'WMO Code',
                    'Country ISO Code', # redundant information
                    'Sample Site Type Name',
                    'Sample Name',
                    'Sample Date Accuracy',
                    'Lith Adj Code',
                    'Lith Adj Name',
                    'Lith Noun Code',
                    'Lith Noun Name',
                    'Min Depth',
                    'Max Depth',
                    'Sampling Method Code',
                    'Sample Media Type Code',
                    'Sample Info',
                    'Observation Method Code',
                    'Observation Method Name',
                    'Measurand Prefix',
                    'Observation Date',
                    'Observation Date Accuracy',
                    'Field Preparation Method Name',
                    'Data Provider Code',
                    'Data Provider Name',
                    'Quality Flag Code',
                    'Quality Flag Name'
                   ]

deutinwater_tritium_simplified = deutinwater_tritium.drop(gnip_unnecessary, axis=1)
deutinwater_tritium_simplified.to_csv('Simplified GNIP', index=False)

## Take average H2 and H3 across each site for each year; Add sites in the same country for the countries total H2 or H3
## Example: Site A is in Country A and has H2 data from 01-2021 to 11-2021, take average of all 11 months
## Site B is in Country A and has H2 data 01-2021 to 11-2021, take average of all 11 months
## Avg Site A + Avg Site B will give us an estimate of the total H2 distributed across the entire Country A
### If Site A has H2 multiple data within the same month, average that too so its representative of the whole month

## CANNOT SUMMATE VALUES BECAUSE THEY ARE RATIOS; FIGURE OUT ALTERNATIVE METHOD 

In [121]:
from pyspark.sql.functions import col, year, month, sum as spark_sum, avg as spark_avg

# read and open the csv as a pyspark dataframe
gnip_simplified = spark.read.csv('Simplified GNIP', header=True, inferSchema=True)

# Convert sample date to timestamp and extract 'year' and 'month'
gnip_simplified = gnip_simplified.withColumn("Sample Date", gnip_simplified["Sample Date"].cast("timestamp"))
gnip_simplified = gnip_simplified.withColumn("Year", year(gnip_simplified["Sample Date"]))
gnip_simplified = gnip_simplified.withColumn("Month", month(gnip_simplified["Sample Date"]))

# use dictionary to map {country : sites}
country_site_mapper = {}
for row in gnip_simplified.select("Project Region", "Sample Site Name").distinct().collect():
    country=row["Project Region"]
    site=row["Sample Site Name"]

    if country in country_site_mapper:
        if site not in country_site_mapper[country]:
            country_site_mapper[country].append(site)

    else:
        country_site_mapper[country]=[site]

# filter for H2 and H3 and designate variables for each
filtered_gnip_h2 = gnip_simplified.filter(col("Measurand Symbol")== "H2")
filtered_gnip_h3 = gnip_simplified.filter(col("Measurand Symbol")== "H3")

# double check filtering worked 
filtered_gnip_h3.select("Measurand symbol").show()

+----------------+
|Measurand symbol|
+----------------+
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
|              H3|
+----------------+
only showing top 20 rows



In [118]:
# Group the data by Sample Site Name, and Year, and aggregate the h2 & h3 average as a new column 
# save in two separate variables

average_h2 = filtered_gnip_h2.groupBy("Sample Site Name", "Year").agg(
    spark_avg("Measurand Amount").alias("avg_h2")
)

average_h3 = filtered_gnip_h3.groupBy("Sample Site Name", "Year").agg(
    spark_avg("Measurand Amount").alias("avg_h3")
)

average_h2.show()

+--------------------+----+-------------------+
|    Sample Site Name|Year|             avg_h2|
+--------------------+----+-------------------+
|            KUMAMOTO|2021| -36.11666666666667|
| INGENIERO JACOBACCI|2021| -41.43333333333333|
|              PATRAS|2021|-27.300000000000004|
|         ALERT (O17)|2021|-230.14285714285714|
|     REYKJAVIK (O17)|2021|-28.833333333333332|
|  RHINE - Diepoldsau|2021| -91.38461538461539|
|             YAOUNDE|2021|-6.1571428571428575|
| ASCENSION IS. (O17)|2021| 10.436363636363636|
|            CHISINAU|2021|              -65.2|
|       MAR DEL PLATA|2021|-22.519999999999996|
|          RIO CUARTO|2021|             -24.97|
|             CACERES|2021|-26.527272727272724|
|          ST. GALLEN|2022|-62.127272727272725|
| CHARLOTTETOWN (O17)|2021|-61.300000000000004|
|                SARH|2021|-16.783333333333335|
|             TBILISI|2021| -48.29090909090908|
|         WALLINGFORD|2021|              -42.5|
|             LOCARNO|2022| -55.03333333

### Manually check if calculation is correct

In [91]:
kuma = filtered_gnip_h2.filter((col("Sample Site Name")== "KUMAMOTO") & (col("Year") == 2021))
kuma_2021_average = kuma.agg(spark_avg("Measurand Amount"))
kuma_2021_average.show()

+---------------------+
|avg(Measurand Amount)|
+---------------------+
|   -36.11666666666667|
+---------------------+



## The calculated average matches, so we can continue

## Map country back to each sample site now that we have the average for each site

In [123]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


# udf (User defined function) allows to apply a function to entire column

def get_country(site):
    for country, sites in country_site_mapper.items():
        if site in sites:
            return country
    return None


get_country_udf = udf(get_country, StringType())
average_h2_mapped = average_h2.withColumn("Country", get_country_udf(col("Sample Site Name")))

average_h2_mapped.show()

+--------------------+----+-------------------+--------------------+
|    Sample Site Name|Year|             avg_h2|             Country|
+--------------------+----+-------------------+--------------------+
|            KUMAMOTO|2021| -36.11666666666667|    Japan (Kumamoto)|
| INGENIERO JACOBACCI|2021| -41.43333333333333|Argentina (variou...|
|              PATRAS|2021|-27.300000000000004|Greece (various s...|
|         ALERT (O17)|2021|-230.14285714285714|               Alert|
|     REYKJAVIK (O17)|2021|-28.833333333333332|           Reykjavik|
|  RHINE - Diepoldsau|2021| -91.38461538461539|         Switzerland|
|             YAOUNDE|2021|-6.1571428571428575|Cameroon (multipl...|
| ASCENSION IS. (O17)|2021| 10.436363636363636|           Ascension|
|            CHISINAU|2021|              -65.2|Rep. of Moldova (...|
|       MAR DEL PLATA|2021|-22.519999999999996|Argentina (variou...|
|          RIO CUARTO|2021|             -24.97|Argentina (variou...|
|             CACERES|2021|-26.527

### Order by country name to see all the sample sites within a country across the years

In [124]:
average_h2_mapped_ordered = average_h2_mapped.orderBy("Country")
average_h2_mapped_ordered.show()

+--------------------+----+-------------------+--------------------+
|    Sample Site Name|Year|             avg_h2|             Country|
+--------------------+----+-------------------+--------------------+
|         ALERT (O17)|2021|-230.14285714285714|               Alert|
|         ALERT (O17)|2022|             -232.2|               Alert|
|              LUANDA|2021|             -23.98|Angola (multiple ...|
|               SUMBE|2021|             -15.15|Angola (multiple ...|
|            LUANDA-P|2021|-18.220000000000002|Angola (multiple ...|
|          NDALATANDU|2021|             -23.45|Angola (multiple ...|
|VERNADSKY (ARGENT...|2021|              -77.4|Antarctica (Verna...|
|       ROTHERA POINT|2021|-127.14166666666665|Antarctica (vario...|
|       EZEIZA (CNEA)|2021|-19.333333333333332|Argentina (multip...|
|          RIO CUARTO|2021|             -24.97|Argentina (variou...|
|          SANTA ROSA|2021|-23.977777777777774|Argentina (variou...|
| INGENIERO JACOBACCI|2021| -41.43

### Understanding the values 

### Negative measurand amounts for H2 (Deuterium) represent isotopic ratios
### In Isotope geochemistry, isotopic ratios are reported relative to a standard reference 
### When they are negative it means the sample has a lower ratio of the heavy isotope (deuterium) compared to the standard
### The values in the table are likely in delta notation, used to express isotopic differences 
### delta = ((R sample / R standard) -1) * 1000
### R is the ratio of the heavy to light isotope (ex: 2H / 1H for deuterium in water) 
### Here, negative delta means the sample has less deuterium relative to the standard 

### Understanding the Units

### ‰ (Permil): Parts per thousand. Common in isotopic measurements to express how much a sample's isotopic ratio deviates from a standard ratio
### VSMOW (Vienna Standard Mean Ocean Water) : Standard reference for water isotopes, particularly deuterium and hydrogen 
### SLAP (Standard Light Antarctic Precipitation) : Another standard used as a reference, particularly for very negative isotopic values
### Permil vs VSMOW-SLAP describes the isotopic deviation from a standard, with the measurement being compared against VSMOW and adjusted using SLAP

In [75]:
# pivot the dataframe to create columns for H2 and H3 
# pivot_gnip = filtered_gnip.groupBy("Sample Site Name", "Year", "Month").pivot("Measurand Symbol").agg(spark_avg("Measurand Amount"))

# rename for clarity 
# pivot_gnip = pivot_gnip.withColumnRenamed("H2", "Average_H2").withColumnRenamed("H3", "Average_H2")

# pivot_gnip.show()

In [ ]:
# ocean_2024=pd.read_csv('cleaned_wod_data.csv') # fix file 
# ocean_2024.head()